## 1. BLIP으로 이미지를 설명하는 프롬프트 1차 생성

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image


def BLIP_caption(image_path):
  # 모델 로드
  processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
  model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

  # 이미지로부터 설명 생성
  inputs = processor(images=Image.open(image_path), return_tensors="pt")

  # 프롬프트 생성
  out = model.generate(**inputs)
  caption = processor.decode(out[0], skip_special_tokens=True)

  return caption

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

a small brown bear standing on a rock in a pond


In [ ]:
# 이미지 열기
image_path = "./example.png"

print(BLIP_caption(image_path))

a cartoon girl holding a cell phone


## 2. GPT를 이용하여 이미지의 프롬프트 구체화

In [ ]:
import base64
import requests

OPENAI_KEY = ""

In [ ]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def get_gpt_response(image_filename, question):
    # Path to your image
    image_path = image_filename

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {OPENAI_KEY}"
    }

    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": question
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 2048
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    return response.json()['choices'][0]['message']['content']

In [ ]:
improved_prompt = get_gpt_response(image_path, 'Generate the detailed prompt that represents the given image.\n'+'Please refer to the following prompt.: \n'+caption)
print(improved_prompt)

A cute cartoon girl taking a selfie with a smartphone. She has long dark hair and is smiling, dressed warmly in a blue and cream varsity jacket with the letter "S" on the sleeve. She's wearing a cozy cream-colored scarf and holding what looks like a round handle or tool in her left hand. The background suggests she is in an indoor setting, possibly wood-paneled, with a small sign behind her that has a hand symbol. The image has a soft, warm color palette and a charming, friendly vibe.


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

response = client.images.generate(
  model="dall-e-3",
  prompt=improved_prompt,
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

im = Image.open(requests.get(image_url, stream=True).raw)
im